In [ ]:
import os
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_val_predict, KFold, train_test_split
from sklearn.metrics import r2_score, make_scorer
from scipy import stats
%matplotlib inline 

# 1. Preparing Data

### Loading HH Data

In [ ]:
hh_data = pd.read_csv("../Data/Intermediate_files/hh_data_2011_cluster_minHH.csv")

In [ ]:
hh_data.head()

### Loading CNN features

In [ ]:
CNN_features = pd.read_csv("../Data/Intermediate_files/google_sat_CNN_features_lsms_ResNet_tf_last.csv")

### Merging

In [ ]:
data=hh_data.merge(CNN_features,on=["i","j"])

In [ ]:
data = data.sample(frac=1, random_state=1783).reset_index(drop=True) #Shuffling the data

In [ ]:
data_features = data[list(set(data.columns) - set(hh_data.columns) - set(['index']))]

In [ ]:
data_features.drop(['Unnamed: 0_x', 'Unnamed: 0_y'], axis=1, inplace=True)

In [ ]:
data_features.shape

### Defining predictors and predicting variables

In [ ]:
y = data["cons"].values #Average normalized consumption per cluster
y = y[y > 0]
y = np.log(y) #Log-normal distribution

In [ ]:
y2 = data["poor_majority"] #Dummy variable for majority of poor in the cluster

In [ ]:
X = data_features.values

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
plt.hist(y)

In [ ]:
pd.DataFrame(X).head()

# 2. Predicting Continuous Indicator

In [ ]:
model=Ridge()

In [ ]:
inner_cv = KFold(n_splits=5, shuffle=True, random_state=1673)
outer_cv = KFold(n_splits=5, shuffle=True, random_state=75788)

In [ ]:
def r2_pearson(ground_truth, predictions):
    r2_pearson=stats.pearsonr(ground_truth, predictions)[0] ** 2
    return r2_pearson
r2_pearson = make_scorer(r2_pearson, greater_is_better=True)

In [ ]:
#Inner cross-validation loop
clf = GridSearchCV(estimator=model, param_grid={'alpha':[0., 0.1,0.01,0.001]}, cv=inner_cv, scoring=r2_pearson)

## From PCA Components

In [ ]:
alphas = np.array([0.01,0.1,1,5,10,20,30,40,50])

In [ ]:
pca = PCA(n_components=10)

In [ ]:
pca.fit(data_features.transpose())

In [ ]:
eigenvectors=pca.components_

In [ ]:
X2 = pd.DataFrame(np.transpose(eigenvectors))
X2.shape

In [ ]:
for i in range(2):
    plt.figure()
    plt.scatter(y, eigenvectors[i],  color='black')

In [ ]:
# Outer Loop with r2 (Pearson)
r2 = cross_val_score(clf, X, y, cv=outer_cv ,scoring=r2_pearson)
print("r2 (pearson): %0.2f (+/- %0.2f)" % (r2.mean(), r2.std() * 2)) 

In [ ]:
# Outer Loop with R2
r2 = cross_val_score(clf, X, y, cv=outer_cv ,scoring='r2')
print("R2: %0.2f (+/- %0.2f)" % (r2.mean(), r2.std() * 2))

In [ ]:
y_hat = cross_val_predict(clf, X2, y, cv=outer_cv)
fig, ax = plt.subplots()
ax.scatter(y, y_hat, edgecolors=(0, 0, 0))
ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k--')
ax.set_xlabel('Log consumption expenditures')
ax.set_ylabel('Model predictions')
plt.show()

## From all (4096) features

In [ ]:
alphas = np.array([1,5,10,20,30,40,50])

In [ ]:
# Outer Loop with r2 (Pearson)
r2 = cross_val_score(clf, X, y, cv=outer_cv ,scoring=r2_pearson)
print("r2 (pearson): %0.2f (+/- %0.2f)" % (r2.mean(), r2.std() * 2))

In [ ]:
r2

In [ ]:
# Outer Loop with R2
R2 = cross_val_score(clf, X, y, cv=outer_cv ,scoring='r2')
print("R2: %0.2f (+/- %0.2f)" % (R2.mean(), R2.std() * 2))

In [ ]:
R2

In [ ]:
neg_mean_squared_error = cross_val_score(clf, X, y, cv=outer_cv,scoring='neg_mean_squared_error')
print("neg_mean_squared_error: %0.2f (+/- %0.2f)" % (-neg_mean_squared_error.mean(), neg_mean_squared_error.std() * 2)) 

In [ ]:
-neg_mean_squared_error

In [ ]:
y_hat = cross_val_predict(clf, X, y, cv=outer_cv)
fig, ax = plt.subplots()
ax.scatter(y, y_hat, edgecolors=(0, 0, 0))
ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k--')
ax.set_xlabel('Log consumption expenditures')
ax.set_ylabel('Model predictions')
plt.show()

In [ ]:
r2_score(y,y_hat)

In [ ]:
stats.pearsonr(y,y_hat)[0] ** 2

# 3. Predicting Dummy Indicator from all features

In [ ]:
model = linear_model.LogisticRegression(penalty="l2")
model.fit(X, y2) 

In [ ]:
cv = KFold(n_splits=10, shuffle=True, random_state=167)

In [ ]:
accuracy = cross_val_score(model, X, y2, cv=cv)
f1 = cross_val_score(model, X, y2, cv=cv,scoring='f1')
precision = cross_val_score(model, X, y2, cv=cv,scoring='precision')
recall= cross_val_score(model, X, y2, cv=cv,scoring='recall')
auc= cross_val_score(model, X, y2, cv=cv,scoring='roc_auc')
confusion= cross_val_score(model, X, y2, cv=cv,scoring='roc_auc')

In [ ]:
scores = cross_val_score(model, X, y2, cv=10,scoring='precision')
accuracy_print=("Accuracy: %0.2f (+/- %0.2f)" % (accuracy.mean(), accuracy.std() * 2))
f1_print=("F1: %0.2f (+/- %0.2f)" % (f1.mean(), f1.std() * 2))
precision_print=("Precision: %0.2f (+/- %0.2f)" % (precision.mean(), precision.std() * 2))
recall_print=("Recall: %0.2f (+/- %0.2f)" % (recall.mean(), recall.std() * 2))
auc_print=("AUC: %0.2f (+/- %0.2f)" % (auc.mean(), auc.std() * 2))
print(os.linesep.join([accuracy_print,f1_print,precision_print,recall_print,auc_print]))

In [ ]:
y2.describe()